In [1]:
import os
from tqdm import tqdm

import cv2
import numpy as np
import matplotlib.pyplot as plt

from utils.dataloader import DataLoader
from utils.vis import MatplotlibVisualizer
from utils.transforms import HairRemoval, Composer
from descriptors.shape import HOGDescriptor
from descriptors.texture import LBPDescriptor, GLCMDescriptor
from utils.segmentation import ThresholdingSegmentation
from descriptors.stats import IntensityStatsGridDescriptor
from descriptors.color import ColorDescriptor, ColorLayoutDescriptor, ColorCooccurrenceMatrixDescriptor

In [2]:
## Classes
CLASSES = ['nevus', 'others']

## Work folfer
work_folder = os.getcwd()
data_folder = os.path.join(work_folder, '..', 'Data/Challenge1')

## Visualizer
matplotlib_visualizer = MatplotlibVisualizer()

In [3]:
transforms_composer = Composer([
                                HairRemoval(),
                                ])
otsu_thresholding = ThresholdingSegmentation(method='otsu')

## Featrure Extraction

### Color Descriptor

In [4]:
modes = ['train', 'val']
## Descriptors
# color_descriptor = ColorDescriptor(bins=(4, 6, 3))
# color_cooccurrence_matrix_descriptor = ColorCooccurrenceMatrixDescriptor(distances=[1], angles=[0, np.pi/2], levels=4)  # Reduce levels and angles
color_layout_descriptor = ColorLayoutDescriptor(grid_x=3, grid_y=3)
intensity_stats_grid_descriptor = IntensityStatsGridDescriptor(grid_x=3, grid_y=3)
color_descriptor = ColorDescriptor(bins=(8, 12, 3))
color_cooccurrence_matrix_descriptor = ColorCooccurrenceMatrixDescriptor(distances=[1], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4], levels=8)
glcm_descriptor = GLCMDescriptor(distances=[1], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4], levels=8, visualize=False, grid_x=3, grid_y=3)
lbp_descriptor = LBPDescriptor(radius=3, n_points=16, grid_x=3, grid_y=3, visualize=False)

for mode in modes:
    ## Make a new folder for the features
    os.makedirs(os.path.join('feautres', mode, 'color'), exist_ok=True)

    ## Data loader
    ### Limit the number of samples to 200 for training and load all samples for validation
    max_samples = None if mode == 'train' else None
    ### Balance the dataset for training
    balance = False if mode == 'train' else False
    dataloader = DataLoader(data_folder, mode, 
                            shuffle=True, 
                            ignore_folders=['black_background', '.DS_Store'], 
                            max_samples=max_samples, 
                            balance=balance,
                            transforms=None, 
                            classes=CLASSES, 
                            mask=False)

    ## Extract features
    features = []
    labels = []
    for i, (img, label, mask, path) in tqdm(enumerate(dataloader), total=len(dataloader), desc=f'Extracting features for {mode}'):
        color_features = color_descriptor.extract(img, mask=None)
        # color_layout_features = color_layout_descriptor.extract(img, mask=None)
        # intensity_stats_grid_features = intensity_stats_grid_descriptor.extract(img, mask=None)
        color_cooccurrence_matrix_features = color_cooccurrence_matrix_descriptor.extract(img, mask=None)
        glcm_features, glcm_img = glcm_descriptor.extract(img, mask=None)
        lbp_features, lbp_img = lbp_descriptor.extract(img, mask=None)
        features.append(np.concatenate([lbp_features, glcm_features, color_features, color_cooccurrence_matrix_features], axis=0))        
        
        ## add label
        labels.append(label)

    ## Save features to disk
    features = np.array(features)
    labels = np.array(labels)
    features_with_labels = np.concatenate([features, labels.reshape(-1, 1)], axis=1)
    np.save(os.path.join('feautres', mode, 'color', 'features.npy'), features_with_labels)

Extracting features for val: 100%|██████████| 3796/3796 [07:41<00:00,  8.23it/s]


## Training

In [5]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import xgboost as xgb

In [54]:
mode = 'train'
# model = SVC(kernel='rbf', C=5.0, random_state=42, degree=5)
# model = xgb.XGBClassifier(objective='binary:logistic', n_estimators=500, learning_rate=0.1, n_jobs=-1)
model= xgb.XGBClassifier(objective='binary:logistic', learning_rate=0.2, n_estimators=1000)


In [55]:
features_with_labels = np.load(os.path.join('feautres', mode, 'color', 'features.npy'))
features = features_with_labels[:, :-1]
labels = features_with_labels[:, -1]

X_train, X_val, y_train, y_val = train_test_split(features, labels, test_size=0.1, random_state=42, stratify=labels)

In [39]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

In [56]:
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.2, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=1000, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

## Validation

In [57]:
y_pred = model.predict(X_val)
print(classification_report(y_val, y_pred, target_names=CLASSES))

              precision    recall  f1-score   support

       nevus       0.86      0.87      0.86       773
      others       0.86      0.85      0.86       747

    accuracy                           0.86      1520
   macro avg       0.86      0.86      0.86      1520
weighted avg       0.86      0.86      0.86      1520



## Testing

In [59]:
# mode = 'val'
features_with_labels_test = np.load(os.path.join('feautres', 'val', 'color', 'features.npy'))
features_test = features_with_labels_test[:, :-1]
labels_test = features_with_labels_test[:, -1]

# features_test = scaler.transform(features_test)
y_pred = model.predict(features_test)
print(classification_report(labels_test, y_pred, target_names=CLASSES))

              precision    recall  f1-score   support

       nevus       0.85      0.86      0.85      1931
      others       0.85      0.84      0.84      1865

    accuracy                           0.85      3796
   macro avg       0.85      0.85      0.85      3796
weighted avg       0.85      0.85      0.85      3796



## Cross Validation

In [66]:
from sklearn.model_selection import cross_val_score, cross_val_predict, StratifiedKFold

# Initialize XGBoost classifier
xgb_clf = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', objective='binary:logistic', n_estimators=500, learning_rate=0.1, n_jobs=-1)

# Define cross-validation strategy (e.g., 5-fold stratified)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Perform cross-validation and get accuracy scores for each fold
cv_scores = cross_val_score(xgb_clf, X_train, y_train, cv=cv, scoring='accuracy')

# Output the results
print("Cross-Validation Accuracy Scores:", cv_scores)
print("Mean CV Accuracy:", cv_scores.mean())
print("Standard Deviation of CV Accuracy:", cv_scores.std())

Cross-Validation Accuracy Scores: [0.8298438  0.82386175 0.80957129 0.83144947 0.82413564]
Mean CV Accuracy: 0.8237723885082341
Standard Deviation of CV Accuracy: 0.007715061318104588


In [67]:
# Cross-validation predictions (optional)
cv_predictions = cross_val_predict(xgb_clf, features_test, labels_test, cv=cv)
print("Classification Report for CV Predictions:\n", classification_report(labels_test, cv_predictions))

Classification Report for CV Predictions:
               precision    recall  f1-score   support

         0.0       0.80      0.78      0.79      1931
         1.0       0.78      0.80      0.79      1865

    accuracy                           0.79      3796
   macro avg       0.79      0.79      0.79      3796
weighted avg       0.79      0.79      0.79      3796



## Grid Search

In [51]:
from sklearn.model_selection import GridSearchCV

# Initialize the base model (XGBClassifier)
model = xgb.XGBClassifier(objective='binary:logistic', n_jobs=-1, random_state=42)

# Define the parameter grid for Grid Search
param_grid = {
    'n_estimators': [700, 1000],  # Number of trees
    'learning_rate': [0.1, 0.2],  # Step size shrinkage
    # 'reg_lambda': [1.0, 0.8],  # L2 regularization term on weights
}

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, 
                        scoring='accuracy',  # Use 'accuracy' as the evaluation metric
                        verbose=4, 
                        n_jobs=-1)  # Parallel processing

# Assuming X_train and y_train are your training data
# Fit the grid search
# scaler_cv = StandardScaler()
# X_train_cv = scaler_cv.fit_transform(features)
X_train_cv = features
y_train_cv = labels
grid_search.fit(X_train_cv, y_train_cv)

# Get the best parameters and best score from the grid search
print("Best Parameters: ", grid_search.best_params_)
print("Best Accuracy: ", grid_search.best_score_)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV 3/5] END learning_rate=0.1, n_estimators=700;, score=0.844 total time= 1.2min
[CV 2/5] END learning_rate=0.1, n_estimators=700;, score=0.839 total time= 1.2min
[CV 1/5] END learning_rate=0.1, n_estimators=700;, score=0.831 total time= 1.2min
[CV 4/5] END learning_rate=0.1, n_estimators=700;, score=0.833 total time= 1.2min
[CV 5/5] END learning_rate=0.1, n_estimators=700;, score=0.854 total time= 1.2min
[CV 2/5] END learning_rate=0.1, n_estimators=1000;, score=0.839 total time= 1.5min
[CV 1/5] END learning_rate=0.1, n_estimators=1000;, score=0.834 total time= 1.5min
[CV 3/5] END learning_rate=0.1, n_estimators=1000;, score=0.843 total time= 1.6min
[CV 1/5] END learning_rate=0.2, n_estimators=700;, score=0.838 total time=  56.2s
[CV 2/5] END learning_rate=0.2, n_estimators=700;, score=0.831 total time=  57.3s
[CV 3/5] END learning_rate=0.2, n_estimators=700;, score=0.843 total time=  57.5s
[CV 4/5] END learning_rate=0.2, n_e

In [52]:
model = grid_search.best_estimator_
# mode = 'val'
features_with_labels_test = np.load(os.path.join('feautres', 'val', 'color', 'features.npy'))
features_test = features_with_labels_test[:, :-1]
labels_test = features_with_labels_test[:, -1]

# features_test = scaler_cv.transform(features_test)
y_pred = model.predict(features_test)
print(classification_report(labels_test, y_pred, target_names=CLASSES))

              precision    recall  f1-score   support

       nevus       0.86      0.86      0.86      1931
      others       0.86      0.85      0.85      1865

    accuracy                           0.86      3796
   macro avg       0.86      0.86      0.86      3796
weighted avg       0.86      0.86      0.86      3796

